In [7]:
import os
import requests
import json
import pandas as pd

In [8]:
# Cities list to gather data from
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
          "Chateau du Haut Koenigsbourg",
          "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon",
          "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence",
          "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege",
          "Toulouse", "Montauban", "Biarritz", "Bayonne",
          "La Rochelle"]

In [ ]:
api_data = {}
API_KEY_2 = '822e12878a17f19f31b1638759f12e0c' # API key for weathermap.org

for city in cities: # Get latitude and longitude for each city in cities list
    # API call
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json")
    print(f"Loading API details for: {city}")
    data = json.loads(response.text)
    api_data[city] = {"lat": data[0]["lat"], "lon": data[0]["lon"]}  # Store latitude and longitude in api_data dict

    for key in api_data:  # Get weather data for each city in api_data dict
        lon = api_data[city]["lon"]
        lat = api_data[city]["lat"]
        
        # API call to weathermap.org 
        params = {
            "lat": lat,
            "lon": lon,
            "appid": API_KEY_2,
            "units": "metric",
            "exclude": "current,minutely,hourly"
        }
        response = requests.get("https://api.openweathermap.org/data/2.5/onecall?", params=params)
        data_weather = json.loads(response.text)

        # Store weather data in api_data dict
        api_data[city] = {
            "lat": lat,
            "lon": lon,
            "avg_temp": sum(day["temp"]["day"] for day in data_weather["daily"]) / len(data_weather["daily"]), # Average temp
            "avg_wind": sum(day["wind_speed"] for day in data_weather["daily"]) / len(data_weather["daily"]), # Average wind speed
            "avg_clouds": sum(day["clouds"] for day in data_weather["daily"]) / len(data_weather["daily"]), # Average cloud cover
            "avg_humidity": sum(day["humidity"] for day in data_weather["daily"]) / len(data_weather["daily"]) # Average humidity
        }

# Save api_data dict in a json file
with open("/Users/pryda/Documents/Projets_Jedha/1_Kayak/cities_weather_v2.json", "w") as f:
    json.dump(api_data, f)

In [9]:
# create dataframe from cities_weather.json
df = pd.read_json("/Users/pryda/Documents/Projets_Jedha/1_Kayak/cities_weather.json", orient="index")
df.head()

,lat,lon,avg_temp,avg_wind,avg_clouds,avg_humidity
Mont Saint Michel,48.635954,-1.511460,5.76625,4.22375,58.125,67.125
St Malo,48.649518,-2.026041,6.16875,5.91625,68.750,68.500
Bayeux,49.276462,-0.702474,5.20625,4.64500,75.125,73.000
Le Havre,49.493898,0.107973,5.29750,5.47250,79.125,72.500
Rouen,49.440459,1.093966,4.79750,3.92375,68.750,73.125


In [10]:
# Adding ranks for cities variables
# Our best cities will have the worst weather conditions
df["avg_temp_rank"] = df.rank(method="first", ascending=False)["avg_temp"]*2 # Rank cities by average temperature (lower is better)
df["avg_wind_rank"] = df.rank(method="first", ascending=True)["avg_wind"]*2 # Rank cities by average wind speed (higher is better)
df["avg_clouds_rank"] = df.rank(method="first", ascending=True)["avg_clouds"]*0.5 # Rank cities by average cloudiness (higher is better)
df["avg_humidity_rank"] = df.rank(method="first", ascending=True)["avg_humidity"]# Rank cities by average humidity (higher is better)
df["total_score"] = df.avg_temp_rank + df.avg_humidity_rank + df.avg_clouds_rank + df.avg_humidity_rank # Total score for each city
df.sort_values(by="total_score", inplace=True, ascending=False) # Sort cities by total score
df.head()

,lat,lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score
Eguisheim,48.044797,7.307962,4.5450,1.84000,86.625,84.750,60.0,6.0,17.5,35.0,147.5
Chateau du Haut Koenigsbourg,48.249490,7.344296,1.6850,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0
Colmar,48.077752,7.357964,4.5100,2.00375,85.500,83.875,62.0,8.0,17.0,33.0,145.0
Dijon,47.321581,5.041470,4.6275,3.39375,82.500,83.750,58.0,24.0,15.0,32.0,137.0
Strasbourg,48.584614,7.750713,4.3625,2.55500,76.875,78.875,66.0,16.0,9.0,27.0,129.0


In [25]:
import plotly.express as px

# create a mapbox of France with cities and their total score
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name=df.index, hover_data=["total_score"],
                        color=df.total_score, zoom=4, height=600, width=800, size=df.total_score, size_max=20)
fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import boto3
session = boto3.Session(profile_name='pryda')
s3 = session.resource('s3')

In [38]:
try:
    bucket = s3.create_bucket(Bucket='projetkayak-dataupload-prod-euwest3', CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})
except Exception as e:
    bucket = s3.Bucket('projetkayak-dataupload-prod-euwest3')

In [53]:
# store df in csv file in the bucket
bucket.Object(key='cities_weather.csv').put(Body=df.to_csv())
# transform hotels.json as csv file and save in the bucket
bucket.Object(key='hotels.csv').put(Body=pd.read_json("/Users/pryda/Documents/Projets_Jedha/1_Kayak/Scrapping/hotelsbooking/hotels.json").to_csv())


{'ResponseMetadata': {'RequestId': 'W79TRSCFCXCM8W6Z',
  'HostId': 'upYNRLMEm4PkgnwzequpE97S2dnfv23ap9URUId1bzJXmaVabmc9P9TdRgaipVlDq/rRXAHhC+w=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'upYNRLMEm4PkgnwzequpE97S2dnfv23ap9URUId1bzJXmaVabmc9P9TdRgaipVlDq/rRXAHhC+w=',
   'x-amz-request-id': 'W79TRSCFCXCM8W6Z',
   'date': 'Sat, 03 Dec 2022 16:58:14 GMT',
   'etag': '"c1b17bea15605166c274b15c990d8361"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c1b17bea15605166c274b15c990d8361"'}

# ETL Process

In [172]:
df_hotels = pd.read_csv("s3://projetkayak-dataupload-prod-euwest3/hotels.csv", index_col=0)
df_cities = pd.read_csv("s3://projetkayak-dataupload-prod-euwest3/cities_weather.csv", index_col=0)

In [173]:
df_hotels.hotel_score = df_hotels.hotel_score.str.replace(",", ".") # Replace "," by "." in hotel_score column
df_hotels.hotel_score = df_hotels.hotel_score.astype(float) # Convert hotel_score column to float
df_hotels = df_hotels.groupby("city").apply(lambda x: x.nlargest(35, "hotel_score")).reset_index(drop=True) # Keep only 25 best hotels per city

In [186]:
df_hotels.head()

,city,hotel_name,hotel_score,hotel_description,hotel_url
0,Aigues Mortes,Nouveau! Magnifique appartement climatisé,10.0,NaN,https://www.booking.com/hotel/fr/nouveau-magni...
1,Aigues Mortes,Au Cœur des Remparts,9.9,NaN,https://www.booking.com/hotel/fr/au-coeur-des-...
2,Aigues Mortes,Marcelle en Camargue,9.9,NaN,https://www.booking.com/hotel/fr/marcelle-en-c...
3,Aigues Mortes,App design au centre des remparts d’Aigues-Mor...,9.9,NaN,https://www.booking.com/hotel/fr/app-design-au...
4,Aigues Mortes,séjour insolite et écologique dans une Tiny house,9.8,NaN,https://www.booking.com/hotel/fr/sejour-insoli...


In [175]:
df_cities.head()

,lat,lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score
Eguisheim,48.044797,7.307962,4.54500,1.84000,86.625,84.750,60.0,6.0,17.5,35.0,147.5
Chateau du Haut Koenigsbourg,48.249490,7.344296,1.68500,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0
Colmar,48.077752,7.357964,4.51000,2.00375,85.500,83.875,62.0,8.0,17.0,33.0,145.0
Dijon,47.321581,5.041470,4.62750,3.39375,82.500,83.750,58.0,24.0,15.0,32.0,137.0
Strasbourg,48.584614,7.750713,4.36250,2.55500,76.875,78.875,66.0,16.0,9.0,27.0,129.0
Besancon,47.238022,6.024362,5.58500,2.67500,81.750,84.125,46.0,18.0,14.0,34.0,128.0
Annecy,45.899235,6.128885,6.34625,1.70875,85.125,81.250,40.0,4.0,16.5,30.0,116.5
Gorges du Verdon,43.749656,6.328562,7.19125,2.54000,83.375,77.875,34.0,14.0,16.0,25.0,100.0
Lille,50.636565,3.063528,4.26000,3.78000,74.500,72.500,68.0,28.0,4.0,14.0,100.0
Lyon,45.757814,4.832011,6.64875,2.92500,83.000,76.125,38.0,20.0,15.5,23.0,99.5


In [176]:
# keep the 5 best cities in df_cities dataframe based on the column total_score
df_cities = df_cities.nlargest(5, "total_score")

In [177]:
df_cities.rename(columns={"lat": "city_lat", "lon": "city_lon"}, inplace=True)

In [178]:
df_cities

,city_lat,city_lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score
Eguisheim,48.044797,7.307962,4.5450,1.84000,86.625,84.750,60.0,6.0,17.5,35.0,147.5
Chateau du Haut Koenigsbourg,48.249490,7.344296,1.6850,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0
Colmar,48.077752,7.357964,4.5100,2.00375,85.500,83.875,62.0,8.0,17.0,33.0,145.0
Dijon,47.321581,5.041470,4.6275,3.39375,82.500,83.750,58.0,24.0,15.0,32.0,137.0
Strasbourg,48.584614,7.750713,4.3625,2.55500,76.875,78.875,66.0,16.0,9.0,27.0,129.0


In [183]:
merged_df = pd.merge(df_cities, df_hotels, left_index=True, right_on="city").reset_index(drop=True)

In [184]:
merged_df.head()

,city_lat,city_lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score,city,hotel_name,hotel_score,hotel_description,hotel_url
0,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,GITE LE COQ ROUGE,9.8,NaN,https://www.booking.com/hotel/fr/gite-le-coq-r...
1,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,Fleur de Vigne,9.7,NaN,https://www.booking.com/hotel/fr/fleur-de-vign...
2,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Grange de Madeleine,9.7,NaN,https://www.booking.com/hotel/fr/la-grange-de-...
3,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,"Le Victoria, Eguisheim, 4-6 p., 4etoiles, réno...",9.7,NaN,https://www.booking.com/hotel/fr/le-victoria-e...
4,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Maison du Rempart,9.6,NaN,https://www.booking.com/hotel/fr/la-maison-du-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Appartement Arcade en hyper centre avec parking,9.3,NaN,https://www.booking.com/hotel/fr/appartement-m...
171,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Bienvenue à Strasbourg - Krutenau,9.3,NaN,https://www.booking.com/hotel/fr/bienvenue-a-s...
172,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Studio de l'Hôtel de Ville,9.3,NaN,https://www.booking.com/hotel/fr/studio-de-l-h...
173,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,The Little Studio,9.3,NaN,https://www.booking.com/hotel/fr/the-little-st...


In [ ]:
from bs4 import BeautifulSoup as BS
import requests  
import pandas as pd  
from time import sleep
from random import randint

for url in merged_df.iterrows():
    sleep(2)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    r = requests.get(url[1]["hotel_url"], headers = headers)
    soup = BS(r.content, 'lxml')
    # get latitude and longitude
    try:
        latlong = soup.find(id="hotel_address").get("data-atlas-latlng").split(",")
    except AttributeError:
        lat, lon = None, None
    hotel_description = soup.find(id="property_description_content").get_text()
    
    lat = latlong[0]
    lon = latlong[1]
       
    merged_df.loc[url[0], "latitude"] = lat
    merged_df.loc[url[0], "longitude"] = lon
    merged_df.loc[url[0], "hotel_description"] = hotel_description
    merged_df.loc[url[0], "hotel_url_check"] = url[1]["hotel_url"]

In [187]:
merged_df

,city_lat,city_lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score,city,hotel_name,hotel_score,hotel_description,hotel_url,latitude,longitude,hotel_url_check
0,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,GITE LE COQ ROUGE,9.8,"\nDoté d'un jardin, d'un barbecue et d'une ter...",https://www.booking.com/hotel/fr/gite-le-coq-r...,48.04170911,7.30569280,https://www.booking.com/hotel/fr/gite-le-coq-r...
1,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,Fleur de Vigne,9.7,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/fleur-de-vign...,48.04611340,7.30516320,https://www.booking.com/hotel/fr/fleur-de-vign...
2,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Grange de Madeleine,9.7,"\nOffrant une vue sur la ville, La Grange de M...",https://www.booking.com/hotel/fr/la-grange-de-...,48.04178290,7.30654670,https://www.booking.com/hotel/fr/la-grange-de-...
3,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,"Le Victoria, Eguisheim, 4-6 p., 4etoiles, réno...",9.7,"\nSitué à Eguisheim, à 5,5 km de la gare de Co...",https://www.booking.com/hotel/fr/le-victoria-e...,48.04254270,7.30744990,https://www.booking.com/hotel/fr/le-victoria-e...
4,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Maison du Rempart,9.6,"\nSituée à Eguisheim, à 6 km de la gare de Col...",https://www.booking.com/hotel/fr/la-maison-du-...,48.04172203,7.30560370,https://www.booking.com/hotel/fr/la-maison-du-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Appartement Arcade en hyper centre avec parking,9.3,\nDoté d'une connexion Wi-Fi et d'un parking p...,https://www.booking.com/hotel/fr/appartement-m...,48.58302680,7.75272710,https://www.booking.com/hotel/fr/appartement-m...
171,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Bienvenue à Strasbourg - Krutenau,9.3,\nLe Bienvenue à Strasbourg - Krutenau propose...,https://www.booking.com/hotel/fr/bienvenue-a-s...,48.57992486,7.76147202,https://www.booking.com/hotel/fr/bienvenue-a-s...
172,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Studio de l'Hôtel de Ville,9.3,\nLe Studio de l'Hôtel de Ville est situé à St...,https://www.booking.com/hotel/fr/studio-de-l-h...,48.58402875,7.74983145,https://www.booking.com/hotel/fr/studio-de-l-h...
173,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,The Little Studio,9.3,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/the-little-st...,48.57019100,7.75086300,https://www.booking.com/hotel/fr/the-little-st...


In [193]:
mask = merged_df["hotel_url_check"] == merged_df["hotel_url"]
merged_df = merged_df[mask]
merged_df.shape

(175, 19)

In [192]:
merged_df

,city_lat,city_lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score,city,hotel_name,hotel_score,hotel_description,hotel_url,latitude,longitude,hotel_url_check
0,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,GITE LE COQ ROUGE,9.8,"\nDoté d'un jardin, d'un barbecue et d'une ter...",https://www.booking.com/hotel/fr/gite-le-coq-r...,48.04170911,7.30569280,https://www.booking.com/hotel/fr/gite-le-coq-r...
1,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,Fleur de Vigne,9.7,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/fleur-de-vign...,48.04611340,7.30516320,https://www.booking.com/hotel/fr/fleur-de-vign...
2,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Grange de Madeleine,9.7,"\nOffrant une vue sur la ville, La Grange de M...",https://www.booking.com/hotel/fr/la-grange-de-...,48.04178290,7.30654670,https://www.booking.com/hotel/fr/la-grange-de-...
3,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,"Le Victoria, Eguisheim, 4-6 p., 4etoiles, réno...",9.7,"\nSitué à Eguisheim, à 5,5 km de la gare de Co...",https://www.booking.com/hotel/fr/le-victoria-e...,48.04254270,7.30744990,https://www.booking.com/hotel/fr/le-victoria-e...
4,48.044797,7.307962,4.5450,1.840,86.625,84.750,60.0,6.0,17.5,35.0,147.5,Eguisheim,La Maison du Rempart,9.6,"\nSituée à Eguisheim, à 6 km de la gare de Col...",https://www.booking.com/hotel/fr/la-maison-du-...,48.04172203,7.30560370,https://www.booking.com/hotel/fr/la-maison-du-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Appartement Arcade en hyper centre avec parking,9.3,\nDoté d'une connexion Wi-Fi et d'un parking p...,https://www.booking.com/hotel/fr/appartement-m...,48.58302680,7.75272710,https://www.booking.com/hotel/fr/appartement-m...
171,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Bienvenue à Strasbourg - Krutenau,9.3,\nLe Bienvenue à Strasbourg - Krutenau propose...,https://www.booking.com/hotel/fr/bienvenue-a-s...,48.57992486,7.76147202,https://www.booking.com/hotel/fr/bienvenue-a-s...
172,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,Studio de l'Hôtel de Ville,9.3,\nLe Studio de l'Hôtel de Ville est situé à St...,https://www.booking.com/hotel/fr/studio-de-l-h...,48.58402875,7.74983145,https://www.booking.com/hotel/fr/studio-de-l-h...
173,48.584614,7.750713,4.3625,2.555,76.875,78.875,66.0,16.0,9.0,27.0,129.0,Strasbourg,The Little Studio,9.3,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/the-little-st...,48.57019100,7.75086300,https://www.booking.com/hotel/fr/the-little-st...


In [196]:
# drop rows with None values in latitude or longitude columns
merged_df = merged_df.dropna(subset=["latitude", "longitude"])
merged_df = merged_df.groupby("city").apply(lambda x: x.nlargest(25, "hotel_score")).reset_index(drop=True)

In [198]:
merged_df.head()

,city_lat,city_lon,avg_temp,avg_wind,avg_clouds,avg_humidity,avg_temp_rank,avg_wind_rank,avg_clouds_rank,avg_humidity_rank,total_score,city,hotel_name,hotel_score,hotel_description,hotel_url,latitude,longitude,hotel_url_check
0,48.24949,7.344296,1.685,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0,Chateau du Haut Koenigsbourg,Villa Grimm,9.8,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/villa-grimm.f...,48.24405000,7.17724400,https://www.booking.com/hotel/fr/villa-grimm.f...
1,48.24949,7.344296,1.685,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0,Chateau du Haut Koenigsbourg,GÎTE **** L'ATELIER DU TONNELIER,9.7,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/gite-l-39-ate...,48.19447500,7.32268400,https://www.booking.com/hotel/fr/gite-l-39-ate...
2,48.24949,7.344296,1.685,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0,Chateau du Haut Koenigsbourg,La Maison de Jeanne,9.6,"\nSitué à Sélestat, à seulement 12 km du châte...",https://www.booking.com/hotel/fr/la-maison-de-...,48.25887983,7.43660423,https://www.booking.com/hotel/fr/la-maison-de-...
3,48.24949,7.344296,1.685,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0,Chateau du Haut Koenigsbourg,La Maison de Vacances,9.5,\nVous pouvez bénéficier d'une réduction Geniu...,https://www.booking.com/hotel/fr/la-maison-de-...,48.35602535,7.41966145,https://www.booking.com/hotel/fr/la-maison-de-...
4,48.24949,7.344296,1.685,1.69625,80.625,82.125,70.0,2.0,13.0,31.0,145.0,Chateau du Haut Koenigsbourg,Les Gîtes d'Adelspach Gîtes & Spa en Alsace,9.4,"\nOffrant une vue sur le jardin, l'établisseme...",https://www.booking.com/hotel/fr/domaine-de-ch...,48.22434218,7.21197235,https://www.booking.com/hotel/fr/domaine-de-ch...


In [199]:
merged_df["city"].value_counts()

Chateau du Haut Koenigsbourg    25
Colmar                          25
Dijon                           25
Eguisheim                       25
Strasbourg                      25
Name: city, dtype: int64